In [ ]:
from typing import List, Dict
import html
import os
from sklearn.metrics import  mean_absolute_percentage_error
import re
import numpy as np
from tqdm import tqdm
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

In [ ]:
POWER = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 600, 700, 800, 900,
         1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 20000, 30000, 40000]

Functions for text preprocessing

In [ ]:
def preprocess_text(text):
    """
    Convert all named and numeric character references (e.g. &gt;, &#62;,
    &x3e;) in the string s to the corresponding unicode characters.
    This function uses the rules defined by the HTML 5 standard
    for both valid and invalid character references, and the list of
    HTML 5 named character references defined in html.entities.html5.
    Args:
        text: Book text.

    Returns:
        Book text in lowercase letters.
    """
    text = html.unescape(text)  # &amp;#x200B; => &#x200B;
    text = html.unescape(text)  # &amp;#x200B; =>
    return text.lower()


def tokenize(text):
    """
    Return the string obtained by replacing the leftmost
    non-overlapping occurrences of the pattern in string by the
    replacement repl.  repl can be either a string or a callable;
    if a string, backslash escapes in it are processed.  If it is
    a callable, it's passed the Match object and must return
    a replacement string to be used.
    Args:
        text: Book text.

    Returns:
        List of words.
    """
    opt = re.sub(r'[^\w\s]', '', text)
    words = opt.split()
    return words

The get_embeddings function returns the emb of words that the glove contains.

The output is a dictionary presented in a 'word': emb format.

The get_tokens function returns a full list of tokens extracted from the text.

In [ ]:
def get_embeddings(path_glove: str) -> Dict[str, np.ndarray]:
    """
    The function returns a dictionary, the key is a word, the value is a vector embedding
    Args:
        path_glove: Path to the glove file.

    Returns:
        The function returns a dictionary Dict[str, np.ndarray], the key is a word, the value is a vector embedding
    """
    embeddings = {}
    with open(path_glove, 'r', encoding='utf-8') as file:

        for line in tqdm(file):
            values = line.split()
            word = values[0]
            embedding = np.asarray(values[1:], dtype='float32')
            embeddings[word] = embedding
    return embeddings


def get_tokens(path_text: str) -> List[str]:
    """
    The function returns a list of tokens from text.
    Args:
        path_text: Path to the text file.

    Returns:
        List of tokens from text.
    """
    tokens = []
    with open(path_text, encoding='utf-8') as file:
        for line in file:
            tokens += tokenize(preprocess_text(line))
    return tokens

The get_tokens_with_embeddings function returns the emb dictionary, which contains only those words that are located in glove.

The get_normalized_embeddings function returns normalized vectors.

The function get_norms_embeddings returns the norms of vectors.

In [ ]:
def get_tokens_with_embeddings(embeddings: np.ndarray, tokens: List, verbose: bool = False) -> np.ndarray:
    """
    The function finds embeddings of words that are in the book.
    Args:
        embeddings: Embeddings.
        tokens: Tokens.
        verbose: Console output of a word that is not in the dictionary glove
        (False do not output to console, True output to console).

    Returns:
        Embeddings of words that are in the book.
    """
    tokens_with_embeddings = []
    for token in tokens:
        try:
            tokens_with_embeddings.append(embeddings[token])
        except KeyError:
            if verbose:
                print('Not found', token)

    return np.asarray(tokens_with_embeddings, dtype='float32')


def get_normalized_embeddings(embeddings: np.ndarray) -> np.ndarray:
    """
    The function finds the weighted average values of embeddings.
    Args:
        embeddings: Embeddings.

    Returns:
        The function returns array weighted average values of embeddings.
    """
    quantity, dim = embeddings.shape
    avg = np.zeros(dim)
    for i in range(quantity):
        avg += embeddings[i]
    avg = avg / quantity

    for i in range(quantity):
        embeddings[i] -= avg

    return embeddings


def get_norms_embeddings(embeddings: np.ndarray) -> np.ndarray:
    """
    The function finds the norm of vectors.
    Args:
        embeddings: Embeddings.

    Returns:
        Norm of word vectors.
    """
    quantity = len(embeddings)
    norms = np.zeros(quantity)

    for i in range(quantity):
        norms[i] = np.linalg.norm(embeddings[i])
    return norms

The get_autocorrelation function calculates autocorrelation.

In [ ]:
def get_autocorrelation(embeddings: np.ndarray, norms: np.ndarray, number_embeddings: int) -> np.ndarray:
    """
    The function finds the autocorrelation values of word vectors
    Args:
        embeddings: Embeddings.
        norms: Norm of word vectors.
        number_embeddings: Number of Embeddings.

    Returns:
        Array of correlation values
    """
    autocorr = np.zeros(len(POWER))
    for k in tqdm(range(len(POWER))):
        sumcorr = 0.
        j = POWER[k]
        for i in range(number_embeddings - j):
            corr = np.dot(embeddings[i], embeddings[i + j]) / norms[i] / norms[i + j]
            sumcorr += corr
        autocorr[k] = sumcorr / (number_embeddings - j)
    return autocorr

The GAPELMAPER class counts metrics.

In [ ]:
class GAPELMAPER:
    """
    Using this class you can find the metric GAPELMAPER.

    Args:
        self.values_x:
        self.values_y: Array of correlation values
        self.negative_index: Index of the first non-negative value after the last index of the negative value.

    """

    def __init__(self, values_x: np.array, values_y: np.array):
        self.negative_index = 0
        self.values_x = values_x
        self.values_y = values_y
        self.y_fitpow = None
        self.y_fitexp = None
        self.y_fitlog = None
        self.__get_negative_index()

    def __get_negative_index(self):
        """The method finds the extreme (maximum) index of a non-negative value"""
        neg = np.where(self.values_y <= 0)[0]
        if neg.size > 0:
            self.negative_index = np.max(neg) + 1

    @staticmethod
    def __mapping(x: np.array, a: float, b: float) -> np.array:
        """
        The method approximates the direct correlation value.

        Args:
            x: the coordinates of any point on the line
            a: is the slope of the line.
            b: is the y-intercept (where the line crosses the y-axis).

        Return:
            Returns an array of numbers.
        """
        return a * x + b

    def get_power_mape(self):
        """
            The method finds the average absolute percentage error for a power law.
        Returns:
            The method returns the average absolute percentage error for a power law.
        """
        values_x = np.log10(self.values_x[self.negative_index:])
        values_y = np.log10(self.values_y[self.negative_index:])

        argspow, _ = curve_fit(self.__mapping, values_x, values_y)
        self.y_fitpow = self.__mapping(values_x, argspow[0], argspow[1])
        return mean_absolute_percentage_error(self.values_y[self.negative_index:], 10 ** self.y_fitpow)

    def get_log_mape(self):
        """
            The method finds the average absolute percentage error for a logarithmic law.
        Returns:
            The method returns the average absolute percentage error for a logarithmic law.
        """
        values_x = np.log10(self.values_x[self.negative_index:])
        values_y = self.values_y[self.negative_index:]

        argslog, _ = curve_fit(self.__mapping, values_x, values_y)
        self.y_fitlog = self.__mapping(values_x, argslog[0], argslog[1])
        return mean_absolute_percentage_error(values_y, self.y_fitlog)

    def get_exp_mape(self):
        """
            The method finds the average absolute percentage error for an exponential law.
        Returns:
            The method returns the average absolute percentage error for an exponential law.
        """
        values_x = self.values_x[self.negative_index:]
        values_y = np.log10(self.values_y[self.negative_index:])

        argsexp, _ = curve_fit(self.__mapping, values_x, values_y)
        self.y_fitexp = self.__mapping(values_x, argsexp[0], argsexp[1])
        return mean_absolute_percentage_error(self.values_y[self.negative_index:], 10 ** self.y_fitexp)

    @staticmethod
    def get_gapelmaper(power_mape, exp_mape):
        """
        Find GAPELMAPER.
        Args:
            power_mape: Average absolute percentage error for a power law.
            exp_mape: Average absolute percentage error for an exponential law

        Returns:
            Metric GAPELMAPER.
        """
        return power_mape / exp_mape

In [ ]:
# Заменить пути на свои!!!
path_glove = 'Glove/multilingual_embeddings.en'
path_text = 'LongText/Don Quijote de la Mancha EN.txt'

_, filename = os.path.split(path_text)

embeddings = get_embeddings(path_glove)
print('Found %s word vectors.' % len(embeddings))
tokens = get_tokens(path_text)
print('All tokens in', filename, len(tokens))
tokens_with_embeddings = get_tokens_with_embeddings(embeddings, tokens, verbose=False)
print("tokens with embeddings", len(tokens_with_embeddings))

normalized_embeddings = get_normalized_embeddings(tokens_with_embeddings)
print('Embeddings normalized', normalized_embeddings.shape, normalized_embeddings.mean())

quantity_embeddings = len(normalized_embeddings)
norms = get_norms_embeddings(normalized_embeddings)
autocorr = get_autocorrelation(normalized_embeddings, norms, quantity_embeddings)
print(filename)

metric = GAPELMAPER(np.array(POWER), autocorr)

print('Power decay')
MAPE_POWER = metric.get_power_mape()
print('MAPE =', MAPE_POWER)
print()
print('Log decay')
MAPE_LOG = metric.get_log_mape()
print('MAPE =', MAPE_LOG)
print()
print('Exp decay')
MAPE_EXP = metric.get_exp_mape()
print('MAPE =', MAPE_EXP)
print()
print('GAPELMAPER')
print(metric.get_gapelmaper(MAPE_POWER, MAPE_EXP))

In [ ]:
class CustomPlot:

    def __init__(self, autocorr):
        self.autocorr = autocorr

    def get_autocorr_plot(self):
        fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15, 15))

        # add data to plots
        ax[0, 0].plot(POWER, self.autocorr, label='corr')
        ax[0, 0].grid(True)
        ax[0, 0].set_xlabel('Log distance', size='large')
        ax[0, 0].set_ylabel('Log Correlation', size='large')
        ax[0, 0].set_yscale('log')
        ax[0, 0].set_xscale('log')

        ax[0, 1].plot(POWER, self.autocorr, label='corr')
        ax[0, 1].grid(True)
        ax[0, 1].set_xlabel('Distance', size='large')
        ax[0, 1].set_ylabel('Log Correlation', size='large')
        ax[0, 1].set_yscale('log')

        ax[1, 0].plot(POWER, self.autocorr, label='corr')
        ax[1, 0].grid(True)
        ax[1, 0].set_xlabel('Log distance', size='large')
        ax[1, 0].set_ylabel('Correlation', size='large')
        ax[1, 0].set_xscale('log')

        ax[1, 1].plot(POWER, self.autocorr, label='corr')
        ax[1, 1].grid(True)
        ax[1, 1].set_xlabel('Distance', size='large')
        ax[1, 1].set_ylabel('Correlation', size='large')
        plt.show()

    @staticmethod
    def plotting_the_graph_correlation(x: np.array, y_original: np.array, y_predict: np.array, logx: str, logy: str):
        """
        The function displays a graph of calculated correlation values,
        and a direct approximation of the correlation values.

        Args:
            x: Array of shifts of the matrix of attention vectors
            y_original: Array of correlation values
            y_predict: An array of values approximating the correlation
            logx:
            logy:
        """
        plt.figure(figsize=(8, 6))
        plt.plot(x, y_original, 'bo', label="y - original")
        plt.plot(x, y_predict, label="y = a * x + b")
        plt.xlabel(logx + r'$\tau$', fontsize=24, fontweight="bold")  # ('𝜏')
        plt.ylabel(logy + r' C($\tau$)', fontsize=24, fontweight="bold")
        plt.legend(loc='best', fancybox=True, shadow=True)
        plt.grid(True)
        plt.show()

    @staticmethod
    def get_discrepancy_plot(values_x: np.ndarray,
                             values_y: np.ndarray,
                             y_fitpow: np.ndarray,
                             y_fitlog: np.ndarray,
                             y_fitexp: np.ndarray,
                             negative_index: int = 0):
        discrep_power = values_y[negative_index:] - 10 ** y_fitpow
        discrep_log = values_y[negative_index:] - y_fitlog
        discrep_exp = values_y[negative_index:] - 10 ** y_fitexp

        fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(14, 4))
        ax[0].plot(values_x[negative_index:], discrep_power, 'bo', label='discrepancy of power fit')
        ax[0].grid(True)
        ax[0].set_xlabel(r'$\tau$', fontsize=24, fontweight="bold")
        ax[0].set_ylabel(r'discrepancy', fontsize=12, fontweight="bold")
        ax[0].set_xscale('log')

        ax[1].plot(values_x[negative_index:], discrep_exp, 'bo', label='discrepancy of exp fit')
        ax[1].grid(True)
        ax[1].set_xlabel(r'$\tau$', fontsize=24, fontweight="bold")
        ax[1].set_ylabel(r'discrepancy', fontsize=12, fontweight="bold")
        ax[1].set_xscale('log')

        ax[2].plot(values_x[negative_index:], discrep_log, 'bo', label='discrepancy of log fit')
        ax[2].grid(True)
        ax[2].set_xlabel(r'$\tau$', fontsize=24, fontweight="bold")
        ax[2].set_ylabel(r'discrepancy', fontsize=12, fontweight="bold")
        ax[2].set_xscale('log')
        plt.show()

In [ ]:
plot_ = CustomPlot(autocorr)
plot_.get_autocorr_plot()

In [ ]:
values_x = np.log10(np.array(POWER)[metric.negative_index:])
values_y = np.log10(autocorr[metric.negative_index:])

plot_.plotting_the_graph_correlation(values_x, values_y, metric.y_fitpow, 'Log',  'Log')

In [ ]:
values_x = np.log10(np.array(POWER)[metric.negative_index:])
values_y = autocorr[metric.negative_index:]

plot_.plotting_the_graph_correlation(values_x, values_y, metric.y_fitlog, 'Log',  '')

In [ ]:
values_x = np.array(POWER)[metric.negative_index:]
values_y = np.log10(autocorr[metric.negative_index:])

plot_.plotting_the_graph_correlation(values_x, values_y, metric.y_fitexp, '',  'Log')

In [ ]:
values_x = np.array(POWER)
values_y = autocorr

plot_.get_discrepancy_plot(values_x, values_y, metric.y_fitpow, metric.y_fitlog, metric.y_fitexp, metric.negative_index)